К текущему моменту мы научились обучать простейшую линейную регрессию: причем научились пользоваться как готовыми методами из sklearn, так и реализовывать поиск решения оптимальных коэффициентов самостоятельно с помощью градиентного спуска.

На последних занятиях мы узнали, что на практике часто приходится прибегать к некоторым хитростям, чтобы оценить, действительно ли модель отлично справляется со своей главной задачей - делать хорошие предсказания на новых данных в будущем, или, как еще говорят, в проде - после запуска.

Для этого было введено понятие так называемой обобщающей способности модели. Оно помогает нам оценить, насколько наша модель подогналась под увиденные данные и насколько она окажется хороша на новых данных.

Обобщающую способность мы научились оценивать с помощью метода Кросс-Валидации.



Построим для старой задачи с предсказанием длительности поездки в такси 2 модели линейной регрессии. Для каждой из них замерим качество на кросс-валидации и на тесте, заранее его отложив.

Считается, что если для K-Fold Кросс-Валидации брать большие значения K (то есть большое количество сплитов для разбиения тренировочной выборки на тренировку-валидацию), то оценка среднеквадратической ошибки будет получаться более справедливой. Так, например, когда K оказывается максимальным, а именно равным количеству объектов в выборке (каждый раз в валидации у нас 1 новый объект), то такой метод Кросс-Валидации еще называют LeaveOneOut.

Главная причина, почему мало где его используют, состоит в дороговизне такого метода. Например, в данной задаче в TAXI_train.csv лежит почти 1,2 млн. объектов. Тогда, чтобы замерить качество на LeaveOneOut валидации, нам пришлось бы обучить 1,2 млн. различных версий одной модели, что, конечно, не вписыватеся в рамки адекватного времени и прочих ресурсов.

Поэтому выберем компромиссное значение в виде 20 фолдов для K-Fold Кросс-Валидации.

Внимание! Для каждой из 2 моделей очевидно было бы справедливым замерять качество на тех же самых объектах, поэтому сплиты лучше зафиксировать сразу.

P.S. Оптимизировать будем MSLE метрику. И сравнивать модели тоже будем по ней будем. Как из домашнего задания про метрики, только без корня! :)

$$\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2$$


### Модель №1 Для начала посчитаем ошибку на Кросс-Валидации и Тесте для нашей самой базовой модели до вычленения каких-либо признаков, а просто взяв все вещественные колонки.

In [1]:
import pandas as pd

In [21]:
initial_data = pd.read_csv('initial_data.csv', index_col='id')

initial_cols = ['vendor_id', 'passenger_count', 'pickup_longitude',
                'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
                'trip_duration']

initial_data = initial_data[initial_cols]

In [52]:
initial_data.shape

(1458644, 7)

### ТрюК!

In [22]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

initial_data = initial_data.assign(log_trip_duration=np.log1p(initial_data['trip_duration']))
initial_data = initial_data.drop('trip_duration', axis=1)


In [23]:
initial_data

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,log_trip_duration
id,,,,,,,
id2875421,2,1,-73.982155,40.767937,-73.964630,40.765602,6.122493
id2377394,1,1,-73.980415,40.738564,-73.999481,40.731152,6.498282
id3858529,2,1,-73.979027,40.763939,-74.005333,40.710087,7.661527
id3504673,2,1,-74.010040,40.719971,-74.012268,40.706718,6.063785
id2181028,2,1,-73.973053,40.793209,-73.972923,40.782520,6.077642
...,...,...,...,...,...,...,...
id2376096,2,4,-73.982201,40.745522,-73.994911,40.740170,6.658011
id1049543,1,1,-74.000946,40.747379,-73.970184,40.796547,6.486161
id2304944,2,1,-73.959129,40.768799,-74.004433,40.707371,6.639876


### Объяснение:

Пусть имеем модель $a(x)$, обученную на MSE от $y$:
$$
\text{MSE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(y_i - a(x_i)\big)^2 \rightarrow min
$$

Также представим модель $a^*(x)$, обученную на MSE от $\log{(y + 1)}$:
$$
\text{MSE}(X, \log{(y+1)}, a^*) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - a^*(x_i)\big)^2 \rightarrow min
$$

Так как вторая модель старается аппроксимироваться ответы в виде $a^*(x) \approx \log{(y+1)}$, то для того, чтобы получить изначальные таргеты, необходимо выходы $a^*(x)$ проэкспоненцировать в виде следующего отношения: $a(x) = e^{a^*(x)} - 1$, то есть $a^*(x) = \log{(a(x) + 1)}$, где $a^*(x)$ дает логарифмические выходы, а $a(x)$ - изначальные. Подставим данное соотношение в MSE выше, получим:

$$
\text{MSE}(X, \log{(y+1)}, \log{(a(x) + 1)}) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

А это в точности MSLE от $y$:

$$
\text{MSLE}(X, y, a) = \frac{1}{\ell}\sum_{i=1}^{\ell} \big(\log{(y_i + 1)} - \log{(a(x_i) + 1)}\big)^2 \rightarrow min
$$

In [24]:
### Выделим test

from sklearn.model_selection import train_test_split

X = initial_data.drop('log_trip_duration', axis=1)
y = initial_data['log_trip_duration']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

In [37]:
### Применим K-Fold на оставшуюяся валидационную часть X_train, y_train

from sklearn.model_selection import KFold, cross_validate

splitter = KFold(n_splits=20, shuffle=True, random_state=33)

In [44]:
### Замерьте качество на кросс-валидации!
from sklearn.linear_model import LinearRegression

losses_train = []
losses_test = []


for train_index, test_index in splitter.split(X_train):
    features_train, features_test = X_train.values[train_index], X_train.values[test_index]
    target_train, target_test = y_train.values[train_index], y_train.values[test_index]
    
    model = LinearRegression()
    model.fit(features_train, target_train)    ### Your code is here
    
    losses_train.append(np.mean((model.predict(features_train) - target_train)**2))
    losses_test.append(np.mean((model.predict(features_test) - target_test)**2))
    
np.mean(losses_train).round(3), np.mean(np.array(losses_test)).round(3)

# model = LinearRegression()
# res = cross_validate(model, X_train, y_train, cv=20, return_estimator=True)

(0.609, 0.613)

In [43]:
### Теперь построим модель на всей тренировочной выборке
### и замерим качество на тесте!
model_lin = LinearRegression()
model_lin.fit(X_train, y_train)    ### Your code is here
np.mean(np.array(model_lin.predict(X_test) - y_test)**2).round(3)

0.606

In [ ]:
### Укажите в ответе на задание 4 среднее качество моделей на валидационных выборках
### и качество модели, обученной на полной тренировочной выборке, на тестовой выборке.
### В качестве разделителя используйте точку, ответ округлите до тысячных.

### Модель №2. Проделаем все то же самое, только для модели с более осознанными признаками, которые удалось получить ранее

In [45]:
processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [50]:
processed_data.head()
processed_data.shape

(1458644, 5)

In [47]:
### Замерять будем MSLE. Можно показать, что для оптимизации MSLE,
### Достаточно логарифмировать таргетную переменную, 
### а потом оптимизировать привычные MSE
import numpy as np

processed_data = processed_data.assign(log_trip_duration=np.log1p(processed_data['trip_duration']))
processed_data = processed_data.drop('trip_duration', axis=1)

In [48]:
X_2 = processed_data.drop('log_trip_duration', axis=1)
y_2 = processed_data['log_trip_duration']

In [53]:
### Важно! Когда сравниваем модели по их качеству
### на валидации и на тесте, не шаффлим данные заново!

test_indexes = X_test.index
train_indexes = X_train.index

X_train_2 = X_2[X_2.index.isin(train_indexes)]
y_train_2 = y_2[y_2.index.isin(train_indexes)]

X_test_2 = X_2[X_2.index.isin(test_indexes)]
y_test_2 = y_2[y_2.index.isin(test_indexes)]

X_train_2.shape

(1166915, 4)

In [28]:
### Замерьте качество на кросс-валидации!
### Your code is here

In [37]:
### Применим K-Fold на оставшуюяся валидационную часть X_train, y_train

from sklearn.model_selection import KFold, cross_validate

splitter = KFold(n_splits=20, shuffle=True, random_state=33)

In [54]:
### Замерьте качество на кросс-валидации!
from sklearn.linear_model import LinearRegression

losses_train = []
losses_test = []


for train_index, test_index in splitter.split(X_train_2):
    features_train, features_test = X_train_2.values[train_index], X_train_2.values[test_index]
    target_train, target_test = y_train_2.values[train_index], y_train_2.values[test_index]
    
    model = LinearRegression()
    model.fit(features_train, target_train)    ### Your code is here
    
    losses_train.append(np.mean((model.predict(features_train) - target_train)**2))
    losses_test.append(np.mean((model.predict(features_test) - target_test)**2))
    
np.mean(losses_train).round(3), np.mean(np.array(losses_test)).round(3)

# model = LinearRegression()
# res = cross_validate(model, X_train, y_train, cv=20, return_estimator=True)

(0.429, 0.431)

In [56]:
### Теперь построим модель на всей тренировочной выборке
### и замерим качество на тесте!
model_lin = LinearRegression()
model_lin.fit(X_train_2, y_train_2)    ### Your code is here
np.mean(np.array(model_lin.predict(X_test_2) - y_test_2)**2).round(3)

0.407

In [6]:
### Теперь построим модель и замерим качество на тесте!
### Your code is here

In [ ]:
### Укажите в ответе на задание 5 среднее качество моделей на валидационных выборках
### и качество модели, обученной на полной тренировочной выборке, на тестовой выборке.
### В качестве разделителя используйте точку, ответ округлите до тысячных.

### Какую модель среди двух стоило бы выбрать? Помогла ли нам базовая обработка признаков с первых уроков? 